# Some maybe usefull stuff:
- Lets keep all the question to Antonio to trello
- The code to mark text: $**<code style="background:yellow;color:blue"> text</code>**$

In [1]:
# pip install -U imbalanced-learn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import streamlit as st
import os

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler



In [3]:
#Getting the data

raw_data = pd.read_csv(r"C:\Users\katar\OneDrive\Dokumenty\Edukacja\Data Science\6_DS_Project\Project\03_Coding\Cards_data\creditcard.csv")

#raw_data = pd.read_csv(r"D:\06_projekt_i_data_science\coding ground\raw data\onlinefraud.csv")

###  ↓↓↓↓↓↓ NOT SURE WHETHER WE NEED THIS FUNCTION ↓↓↓↓↓↓

In [4]:
def data_preprocessing(raw_data: pd.DataFrame, hour: bool = True, feature_list: list = [], labels:bool = True):
    ''' A function to preprocess raw data:
    - scaling: True/False
    - balance: over-sampling method or None
    - hour: True/Falce
        whether the 'Time' feature should be changed to hours OBS!! The 'Time' column is dropped automaticaly
    - feature_list: list of features which should be droped 
    - label: True/False if True the labels will be splited to a new variable.
   '''

    data = raw_data
    labels_data = pd.Series(dtype = int)
    
# Spliting labels from the other features (if the raw_data includes column 'Class'):
    if labels == True:
        if 'Class' in data.columns:
            labels_data = data['Class']
            data = data.drop('Class', axis=1)
        else:
            raise ValueError("No target variable 'Class' found in the dataset")


# Changing the 'Time' feature into 'hour':
    # I calculate floor to get hours. As a result I get values from 0 to 47. Next I calulate modulo 24 -> I got values from 0 to 23. Last step adding 1 to get values from 1 to 24.
    if 'Time' in data.columns:
        if hour == True:
            data['hour'] = ((((data['Time']) // (60 * 60)) % 24) + 1)
            data['hour'] = data['hour'].astype(int)
            data = data.drop('Time', axis=1)
            
            # one-hot-encoded dataset
            data = pd.get_dummies(data, columns = ['hour'], prefix='hour')
            
    else:
        print("No 'Time' variable in the dataset")

# Dropping not needed features based on the feature_list
    if feature_list:
        data = data.drop(feature_list, axis=1)
        

    return labels_data, data

In [5]:
y,X = data_preprocessing(raw_data, hour = True, labels = True)

### ↓↓↓↓↓↓ The CLASS starts here ↓↓↓↓↓↓

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin


class DataCleaner(BaseEstimator, TransformerMixin):
    '''The DataCleaner class is a custom transformer that can be used to clean and preprocess data in a machine learning pipeline.

    The class has several parameters:
        - hour: a boolean parameter that indicates whether to convert the Time feature to hours and one-hot-encode the resulting hour feature (default True)
        - feature_to_drop: a list of feature names to drop from the DataFrame (default [])
        - labels: a boolean parameter that indicates whether to split the Class feature (if it exists) from the other features and store it separately (default True)
        - imputer: an imputer object from scikit-learn to use for imputing missing values in the DataFrame (default None)
        - scaler: a scaler object from scikit-learn to use for scaling the numerical features in the DataFrame (default None)

    
    The class implements two methods, fit and transform.    '''

    def __init__(self, hour:bool =True, feature_to_drop:list =[], labels:bool =True, imputer:str = None, scaler:str = None):
        self.hour = hour
        self.feature_to_drop = feature_to_drop
        self.labels = labels
        self.imputer = imputer
        self.scaler = scaler
        self.labels_data = None
        self.data_num = ['Amount']
        
    def fit(self, data, y=None):
        if self.imputer:
           self.imputer.fit(data)
        
        if self.scaler:
            self.scaler.fit(data.loc[:,self.data_num])
        
        return self #, self.imputer, self.scaler
    
    def transform(self, data, y=None):

        #Imputing missing_values 
        if data.isnull().values.any() or data.isna().values.any():
            data = pd.DataFrame(self.imputer.transform(data), columns=data.columns)
                
       
        # Spliting labels from other features (if the data includes column 'Class'):
        if self.labels:
            if 'Class' in data.columns:
                self.labels_data = data['Class']
                data = data.drop('Class', axis=1)
                
            else:
                raise ValueError("No target variable 'Class' found in the dataset")
        labels_data = self.labels_data

        if self.scaler:
            data_num_scaled = self.scaler.transform(data.loc[:,self.data_num])
            data[self.data_num] = data_num_scaled


        # Converting 'Time' to 'hours':
        if self.hour:
            if 'Time' in data.columns:
                data['hour'] = ((((data['Time']) // (60 * 60)) % 24) + 1)
                data['hour'] = data['hour'].astype(int)
                data = data.drop('Time', axis=1)
            
                # one-hot-encoded dataset
                data = pd.get_dummies(data, columns=['hour'], prefix='hour')
            else:
                print("No 'Time' variable in the dataset")

        # Dropping not needed features based on the feature_to_drop list
        if self.feature_to_drop:
            data = data.drop(self.feature_to_drop, axis=1)
        
        
        return data, labels_data

In [15]:
raw_data.at[0, 'V1']=None
raw_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,NaN,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [23]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

imputer = SimpleImputer(strategy='most_frequent')
scaler = MinMaxScaler(feature_range=(-1,1))
clean = DataCleaner(hour=False, feature_to_drop=['V1', 'V2'], labels=True,  imputer = imputer, scaler=scaler)
clean.fit(raw_data)


DataCleaner(feature_to_drop=['V1', 'V2'], hour=False,
            imputer=SimpleImputer(strategy='most_frequent'),
            scaler=MinMaxScaler(feature_range=(-1, 1)))

In [24]:
X_train, y_train = clean.transform(raw_data)

In [25]:
X_train.head()

,Time,V3,V4,V5,V6,V7,V8,V9,V10,V11,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,0.762730,-0.373955,0.526877,-0.464663,-0.466370,0.572888,-0.049377,0.021201,-0.495031,...,0.165884,0.122369,0.045984,0.327586,-0.217495,0.170244,-0.210886,-0.162048,-0.374607,-0.988352
1,0.0,0.680597,-0.456407,0.532241,-0.475617,-0.470249,0.572597,-0.092038,0.010535,-0.237625,...,0.159060,0.115680,-0.039526,0.333876,-0.327120,0.174581,-0.107974,-0.167310,-0.373155,-0.999791
2,1.0,0.736282,-0.462469,0.524658,-0.437756,-0.459646,0.576085,-0.178795,0.026036,-0.355156,...,0.171710,0.130955,0.092060,0.357878,-0.421292,0.119030,-0.194546,-0.169021,-0.376177,-0.970522
3,1.0,0.736967,-0.572678,0.531294,-0.448882,-0.466394,0.578868,-0.170001,0.015170,-0.456365,...,0.156100,0.119467,0.020554,0.325214,-0.552348,0.228491,-0.221607,-0.164663,-0.371258,-0.990386
4,2.0,0.728501,-0.460407,0.525950,-0.472032,-0.462064,0.564967,-0.018101,0.048606,-0.527291,...,0.169231,0.122655,0.094541,0.326784,-0.197460,0.132685,0.014994,-0.158878,-0.365020,-0.994551


In [22]:
print(y_train)

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
284802    0.0
284803    0.0
284804    0.0
284805    0.0
284806    0.0
Name: Class, Length: 284807, dtype: float64
